## TensorBoard

우리가 사용할 수 있는 강력한 시각화 툴입니다.

In [203]:
import keras
import tensorflow as tf
from zipfile import ZipFile
from pandas import read_csv
from pathlib import Path
import io
with ZipFile("../Data/고객 대출등급 분류 해커톤.zip","r") as zpf:
    data=io.BytesIO(zpf.read("고객 대출등급 분류 해커톤/train.csv"))

data=read_csv(data)

이전에 사용했던 Selu를 이용한 모델링은 다시 구현해봅시다

In [205]:
data.head(4)

,ID,대출금액,대출기간,근로기간,주택소유상태,연간소득,부채_대비_소득_비율,총계좌수,대출목적,최근_2년간_연체_횟수,총상환원금,총상환이자,총연체금액,연체계좌수,대출등급
0,TRAIN_00000,12480000,36 months,6 years,RENT,72000000,18.90,15,부채 통합,0,0,0.0,0.0,0.0,C
1,TRAIN_00001,14400000,60 months,10+ years,MORTGAGE,130800000,22.33,21,주택 개선,0,373572,234060.0,0.0,0.0,B
2,TRAIN_00002,12000000,36 months,5 years,MORTGAGE,96000000,8.60,14,부채 통합,0,928644,151944.0,0.0,0.0,A
3,TRAIN_00003,14400000,36 months,8 years,MORTGAGE,132000000,15.09,15,부채 통합,0,325824,153108.0,0.0,0.0,C


In [206]:
import numpy as np
num_data=tf.data.Dataset.from_tensor_slices(data.select_dtypes(np.number))
label_data=tf.data.Dataset.from_tensor_slices(data[["대출등급"]])

In [207]:
# 정규화

normalize=keras.layers.Normalization()
normalize.adapt(num_data)

# 라벨 텐서처리

stringlookup=keras.layers.StringLookup(max_tokens=data["대출등급"].nunique(),num_oov_indices=0)
stringlookup.adapt(label_data)
cate_emb=keras.layers.CategoryEncoding(num_tokens=len(stringlookup.get_vocabulary()))

proceed_label_data=label_data.map(lambda value : 
               cate_emb(stringlookup(value)))

2025-01-20 15:31:38.415280: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
2025-01-20 15:31:52.180162: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [208]:
dataset=tf.data.Dataset.zip((num_data,proceed_label_data))

In [209]:
for i in dataset.take(1):
    print(i)

(<tf.Tensor: shape=(9,), dtype=float64, numpy=
array([1.248e+07, 7.200e+07, 1.890e+01, 1.500e+01, 0.000e+00, 0.000e+00,
       0.000e+00, 0.000e+00, 0.000e+00])>, <tf.Tensor: shape=(7,), dtype=float32, numpy=array([0., 1., 0., 0., 0., 0., 0.], dtype=float32)>)


2025-01-20 15:31:52.213518: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [210]:
from functools import partial
kwargs={"activation" : "selu","kernel_initializer":"lecun_normal","bias_initializer":"lecun_normal"}
base_layer=partial(
    keras.layers.Dense,**kwargs
)

inputs=keras.layers.Input(shape=(9,))

normal_input=normalize(inputs)

for _ in range(10):
    normal_input=base_layer(128)(normal_input)

outputs=keras.layers.Dense(7,activation="softmax",kernel_initializer="lecun_normal",
                          bias_initializer="lecun_normal")(normal_input)

model=keras.Model(inputs,outputs)

In [211]:
model.summary()

Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_4 (InputLayer)      │ (None, 9)              │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ normalization_4 (Normalization) │ (None, 9)              │            19 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_35 (Dense)                │ (None, 128)            │         1,280 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_36 (Dense)                │ (None, 128)            │        16,512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_37 (Dense)                │ (None, 128)            │        16,512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_38 (Dense)                │ (None, 128)            │        16,512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_39 (Dense)                │ (None, 128)            │        16,512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_40 (Dense)                │ (None, 128)            │        16,512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_41 (Dense)                │ (None, 128)            │        16,512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_42 (Dense)                │ (None, 128)            │        16,512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_43 (Dense)                │ (None, 128)            │        16,512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_44 (Dense)                │ (None, 128)            │        16,512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_45 (Dense)                │ (None, 7)              │           903 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 150,810 (589.11 KB)

 Trainable params: 150,791 (589.03 KB)

 Non-trainable params: 19 (80.00 B)

In [212]:
#comple

learning_schedule=keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=0.01,decay_rate=0.1,decay_steps=10
)
nesterov=keras.optimizers.SGD(learning_rate=learning_schedule,momentum=0.9,nesterov=True)

loss=keras.losses.CategoricalCrossentropy()
metric=[keras.metrics.CategoricalAccuracy()]

model.compile(optimizer=nesterov,loss=loss,metrics=metric)

In [213]:
# early_stop

early_stop=keras.callbacks.EarlyStopping(patience=2,restore_best_weights=True)

### Tensorboard 사전준비

텐서보드는 이벤트 파일이라는 이진로그파일이 필요합니다.

다행히 이 파일의 작성은 `keras.callbacks.TensorBoard`가 지원하고 있습니다.

다만, 일반적으로 이벤트파일은

텐서보드 서버는 루트 디렉토리를 가르키고, 프로그램은 실행할때마다 다른 서브 디렉토리에 이벤트를 기록하게끔 합니다.

이를 위한 함수를 하나 작성하죠

In [214]:
from time import strftime
def root_path(path:str):
    def get_logs():
        return Path(path)/strftime("run_%Y_%m_%d_%H_%M")
    return get_logs

In [215]:
tb=keras.callbacks.TensorBoard("my_logs",profile_batch=(10,100))

callback=[early_stop,tb]

2025-01-20 15:31:52.323417: I external/local_tsl/tsl/profiler/lib/profiler_session.cc:104] Profiler session initializing.
2025-01-20 15:31:52.323425: I external/local_tsl/tsl/profiler/lib/profiler_session.cc:119] Profiler session started.
2025-01-20 15:31:52.323647: I external/local_tsl/tsl/profiler/lib/profiler_session.cc:131] Profiler session tear down.


In [216]:
for i in dataset.take(1):
    print(i)

(<tf.Tensor: shape=(9,), dtype=float64, numpy=
array([1.248e+07, 7.200e+07, 1.890e+01, 1.500e+01, 0.000e+00, 0.000e+00,
       0.000e+00, 0.000e+00, 0.000e+00])>, <tf.Tensor: shape=(7,), dtype=float32, numpy=array([0., 1., 0., 0., 0., 0., 0.], dtype=float32)>)


2025-01-20 15:31:52.348744: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [217]:
copy_dataset=dataset
dataset=dataset.batch(16)

In [218]:
train_size=int(len(dataset)*0.9)
train=dataset.take(train_size)

other=dataset.skip(train_size)

valid_size=int(len(other)*0.5)
valid=other.take(valid_size)

test=other.skip(valid_size)

In [219]:
#train=train.repeat(3).shuffle(buffer_size=train_size*3).batch(128).prefetch(1)

In [220]:
model.predict(valid.map(lambda x,y:x))

301/301 ━━━━━━━━━━━━━━━━━━━━ 4s 7ms/step


array([[0.2703163 , 0.14151224, 0.23559584, ..., 0.06176174, 0.19822265,
        0.02810469],
       [0.10289913, 0.11085254, 0.043907  , ..., 0.13035712, 0.40300876,
        0.04152953],
       [0.03983286, 0.0457298 , 0.04272838, ..., 0.08412149, 0.00725886,
        0.7699036 ],
       ...,
       [0.17157638, 0.04311217, 0.05015269, ..., 0.04311013, 0.08916571,
        0.52887934],
       [0.07250931, 0.08985303, 0.05098843, ..., 0.10779391, 0.06021112,
        0.6102977 ],
       [0.02784516, 0.01229138, 0.06906776, ..., 0.06477381, 0.00297892,
        0.77397376]], dtype=float32)

In [221]:
model.fit(train,epochs=10,callbacks=callback,validation_data=valid)

Epoch 1/10
  18/5417 ━━━━━━━━━━━━━━━━━━━━ 2:00 22ms/step - categorical_accuracy: 0.1738 - loss: 2.3043   

2025-01-20 15:31:57.778726: I external/local_tsl/tsl/profiler/lib/profiler_session.cc:104] Profiler session initializing.
2025-01-20 15:31:57.778737: I external/local_tsl/tsl/profiler/lib/profiler_session.cc:119] Profiler session started.


 103/5417 ━━━━━━━━━━━━━━━━━━━━ 2:02 23ms/step - categorical_accuracy: 0.2771 - loss: 1.9035

2025-01-20 15:31:59.782670: I external/local_tsl/tsl/profiler/lib/profiler_session.cc:70] Profiler session collecting data.
2025-01-20 15:31:59.862296: I external/local_tsl/tsl/profiler/lib/profiler_session.cc:131] Profiler session tear down.
2025-01-20 15:31:59.862497: I external/local_tsl/tsl/profiler/rpc/client/save_profile.cc:144] Collecting XSpace to repository: my_logs/train/plugins/profile/2025_01_20_15_31_59/ijeonghun-ui-MacBookPro.local.xplane.pb


5417/5417 ━━━━━━━━━━━━━━━━━━━━ 120s 22ms/step - categorical_accuracy: 0.3307 - loss: 1.6824 - val_categorical_accuracy: 0.3378 - val_loss: 1.6558
Epoch 2/10
5417/5417 ━━━━━━━━━━━━━━━━━━━━ 118s 22ms/step - categorical_accuracy: 0.3332 - loss: 1.6713 - val_categorical_accuracy: 0.3378 - val_loss: 1.6558
Epoch 3/10
5417/5417 ━━━━━━━━━━━━━━━━━━━━ 116s 21ms/step - categorical_accuracy: 0.3332 - loss: 1.6713 - val_categorical_accuracy: 0.3378 - val_loss: 1.6558


vs code 에서 상단의 `Python : launch TensorBoard`를 활성화하면 됩니다